In [3]:
import pandas as pd
from glob2 import glob
import os
from IPython.display import display, HTML
from tqdm import tqdm

def show_df(df):
    if isinstance(df, pd.Series):
        display(HTML(df.to_frame().to_html()))
    else:
        display(HTML(df.to_html()))
    print("table shape:", df.shape)


def extract_head(df):
    heads = df.iloc[3].tolist()
    name = df.columns[0]
    for i,  h in enumerate(heads):
        if str(h)== 'nan':
            heads[i] = df.iloc[4,i]
    heads[0] = name + " "+ heads[0] + " " + df.iloc[2,0]
    return heads


def extract_row(df, kw):
    res = []
    for i, cls in enumerate(df.iloc[:,0].tolist()):
        if str(cls).find(kw)!=-1:
            res.append( df.iloc[i] )
    if len(res)==0:
        return None
    return pd.concat(res, axis=0)


#     heads = extract_head(df)
#     df.columns = heads
#     r = extract_row(df, "橡胶").to_frame().T


# shengfen = [os.path.split(t)[-1] for t in glob(r'E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\*')]
shengfen = ['北京']
# shengfen = list(set(shengfen)-set(['山东']))
root_path = r"E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴"
save_path = "F:\\nianjianjieguo"


In [18]:
'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\E225.xlsx'

'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\E225.XLS'

In [27]:
cnt = 0

for sf in tqdm(shengfen):
    path_sf = os.path.join(root_path, sf)
#     files = glob(os.path.join(path_sf, '**', '*.xls*'))
    files = glob(os.path.join(path_sf, '*','xls', '*.xls*'))
    years_list = list(set([ t.split('\\')[-3] for t in files ]))
    
    for year in tqdm(years_list):
        files = glob(os.path.join(path_sf, year, '**', "*.xls*"))
    
        res = {}
        bad = []

        for f in files:
            try:
                xl = pd.ExcelFile(f)
                sheet_names = xl.sheet_names
            except:
#                 print("bad read:", f)
                bad.append(f)
                continue
            name = os.path.split(f)[-1].split('.')[0]

            for sheet in sheet_names:
                df = xl.parse(sheet)
                if (df.shape[0]<10): continue
                ind = df.iloc[:,0].map(lambda t: str(t).find('橡胶')!=-1 or str(t).find('塑料')!=-1 or str(t).find('轮胎')!=-1 )
                if (ind.sum()==0): continue
#                 print("find {} {} {}".format(sf, year, name))
                sel = df.loc[ind]
                header = df.iloc[:10,:]
                mer = pd.concat( [header, sel], axis=0 )
                res[name] = mer
        
        if not os.path.exists(os.path.join(save_path, sf)):
            os.mkdir(os.path.join(save_path, sf))
        
        writer = pd.ExcelWriter(os.path.join(save_path, sf, "{}.xlsx".format(year)))
        sheet_names = []
        for k,v in res.items():
            name = str(v.columns[0])
            if len(name)>31: name = name[:31]
            if name=='nan' or len(name)==0 or name in sheet_names:
                name = k
                if len(name)>31: name = name[:28]
                if name in sheet_names: 
                    name = name + "_" + str(cnt)
                    cnt += 1
            sheet_names.append(name)
            v.to_excel(writer, sheet_name=name)

        writer.save()
        with open( os.path.join(save_path, sf, "{}_bad_read.txt".format(year)), "w") as fd:
            for t in bad:
                fd.write(t+"\n")


100%|███████████████████████████████████████████| 1/1 [12:47<00:00, 767.82s/it]


In [135]:
# path = r"E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls"
# files = glob(os.path.join(path, "*.xls"))
test_files = glob(r"E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\**\*.xls")
test_files

6932


['E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0101.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0102.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0103.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0104.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0105.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0106.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0107.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0107X.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0107X1.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0107X2.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0107X3.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0107X4.xls',
 'E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2011\\xls\\C0108.xls',
 'E:\\孙宗帝资料\\202

In [15]:
# res = {}
# bad = []
# for f in test_files:
#     try:
#         xl = pd.ExcelFile(f)
#         sheet_names = xl.sheet_names
#     except:
#         print("bad read:", f)
#         bad.append(f)
#         continue
#     name = os.path.split(f)[-1].split('.')[0]
    
#     for sheet in sheet_names:
#         df = xl.parse(sheet)
#         if (df.shape[0]<10): continue
#         ind = df.iloc[:,0].map(lambda t: str(t).find('橡胶')!=-1 or str(t).find('塑料')!=-1 or str(t).find('轮胎')!=-1 )
#         if (ind.sum()==0): continue
#         sel = df.loc[ind]
#         header = df.iloc[:10,:]
#         mer = pd.concat( [header, sel], axis=0 )
#         res[name] = mer

bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\C0428.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\c1410x1.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\c1912.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\E0428.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\E1315.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\e1410x1.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2014\xls\e1912.xls


In [16]:
for k,v in res.items():
    print("table name:", k)
    show_df(v)
    print("-------------------------------------------------------------------------------------")

table name: C0506


,5-6 按行业分的固定资产投资(2013年),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,单位:万元
2,类 别,固定资产\n投资额,建设总投资,施工项目\n(个),NaN
3,NaN,NaN,NaN,NaN,新开工\n项 目
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,总 计,358758558,590718907,37496,28823
9,(一)农、林、牧、渔业,8309643,11749943,1808,1469


table shape: (11, 5)
-------------------------------------------------------------------------------------
table name: C0510


,5-10 新增生产能力(2013年),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,能力名称,建设规模\n,本年施\n工规模\n,NaN,累计新增\n生产能力,NaN
3,NaN,NaN,NaN,新开工\n能力,NaN,本年新\n增能力
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,原煤开采 (万吨/年),2207,2129,645,803,758
8,洗 煤 (万吨/年),1117.4,986.4,721.4,961.4,743.4
9,焦 炭 (万吨/年),1395,1377.4,1295,1378.8,1377.4


table shape: (11, 6)
-------------------------------------------------------------------------------------
table name: C0510X1


...

table shape: (11, 4)
-------------------------------------------------------------------------------------
table name: c2013


,20-13 制造业各大类行业产品质量合格率(2013年),Unnamed: 1
0,NaN,NaN
1,NaN,NaN
2,类别,产品质量\n合格率(%)\n
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,农副食品加工业,82.9
7,食品制造业,96.2
8,酒、饮料和精制茶制造业,94.2
9,烟草制品业,100


table shape: (11, 2)
-------------------------------------------------------------------------------------
table name: c2014


,20-14 产品质量监督抽查情况(2013年),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,类 别,监督检验\n企 业 数\n(个)\n,检验批次\n(批次)\n,合格批次\n(批次)\n,批 次\n合格率\n(%)\n
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


table shape: (14, 5)
-------------------------------------------------------------------------------------


In [107]:
# writer = pd.ExcelWriter("2016_山东.xlsx")
# sheet_names = []
# for k,v in res.items():
#     name = str(v.columns[0])
#     if name=='nan' or len(name)==0 or name in sheet_names:
#         name = k
#     if len(name)>31: name = name[:31]
#     sheet_names.append(name)
#     v.to_excel(writer, sheet_name=name)

# writer.save()

In [6]:
# pd.read_excel(f, sheet_name='A0701')
# pd.read_excel('E:\\孙宗帝资料\\2020年博士毕业相关资料\\论文年鉴资料\\省份年鉴\\山东\\山东统计年鉴2016\\xls\\C0803.xls')

In [5]:
# columns={}
# df=pd.read_excel(r"E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\C0405.xls")
# df.head(20)

# table_name = os.path.split(r"E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\C0417X.xls")[-1].split('.')[0]
# # table_name
# # if table_name.find('X') != -1:
# #         table_name = table_name[:table_name.find('X')]
# table_name[:-3]
# df.shape[0]
# col=[]
# for i in range(df.shape[1]):
#             for j in range(df.shape[0]):
#                 if str(df.iloc[j,i])!='nan':
#                     col.append(str(df.iloc[j,i]).replace('\n', ''))
#                     break
# col
# df.columns[0]
# # df.head(20)
# # df.iloc[0,0]
# # col[0]
# # col[0]=df.columns[0]+col[0]
# # col[0]
# # col
# columns[table_name]=col
# columns
# res = {}
# name_1=[]

# for i in range(df.shape[0]):
#     name=str(df.iloc[i,0])
#     name_1.append(str(df.iloc[i,0]).replace('\n',''))
#     if name.find('全省合计')!=-1 or name.find('青岛')!=-1 or name.find('轮胎')!=-1:
#         if table_name in res:
#             res[table_name].append(df.iloc[i])
#         else:
#             res[table_name] = [df.iloc[i]]
# #     col.append(str(df.iloc[j,i]).replace('\n', ''))
# name_1
# # [res['C0417X'].to_frame()]

# # mer = pd.concat([res.to_frame().T ], axis=0)
# res.keys()




In [4]:
# file="C0110.xls
# df=pd.read_excel(file)
# df.head(20)

In [3]:
# file = "05-06.xls"
# df = pd.read_excel(files[0])
# df.head(20)
# # files[0]
# # for ff in files:
# #     print(pd.ExcelFile(ff).sheet_names)
# xl = pd.ExcelFile(f)
# sheet_names = xl.sheet_names
# xl.parse(sheet_names[0]).shape

In [62]:
# res = {}
# bad = []
# columns = {}
# for f in files:
#     try:
#         df = pd.read_excel(f)
#     except:
#         print("bad read:", f)
#         bad.append(f)
#         continue
#     table_name = os.path.split(f)[-1].split('.')[0]
    
#     if table_name.find('X') != -1:
#         table_name = table_name[:table_name.find('X')]
#         col = []
#         for i in range(df.shape[1]):
#             for j in range(df.shape[0]):
#                 if str(df.iloc[j,i])!='nan':
#                     col.append(str(df.iloc[j,i]).replace('\n', ''))
#                     break
# #         print(table_name)
#         try:
#             col[0]=str(df.columns[0])+" "+col[0]
#         except:
#             pass
       
#         columns[table_name] = col
        
#     for i in range(df.shape[0]):
#         name = str(df.iloc[i,0])
#         if name.find('橡胶')!=-1 or name.find('塑料')!=-1 or name.find('轮胎')!=-1:
#             if table_name in res:
#                 res[table_name].append(df.iloc[i])
#             else:
#                 res[table_name] = [df.iloc[i]]

       

bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\C0428.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\C1410X.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\C1503.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\C1912.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\E0428.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\E1410X.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\E1503.xls
bad read: E:\孙宗帝资料\2020年博士毕业相关资料\论文年鉴资料\省份年鉴\山东\山东统计年鉴2016\xls\E1912.xls


In [63]:
table_name

'xsjm'

In [64]:
res2 = []                
for k in res.keys():
    mer = pd.concat([t.to_frame().T for t in res[k]], axis=0)
    try:
        mer.columns = columns[k]
    except:
        pass
    res2.append(mer) 